# 확률 네트워크기반 친환경 자동차 충전소(GVCS) 로케이션 인텔리전스(Location Intelligence) 대시보드

## 제안 배경

**친환경 자동차 생산에만 관심이 쏠려 사용자를 고려하지 못한 설치로 인한 잉여 충전소 발생**

- 민관 협력을 통해 충전 서비스 관련 사업이 꾸준히 증가함. 하지만 전국에 설치된 공공 급속충전기 총 2,896개(2021년 기준) 중 하루 1회 이하로 사용한 충전기가 1,164개로 전체의 40.1%를 차지하고 있다. 심지어 17개는 전혀 사용하지 않은 것으로 나타남.

- 이와 같은 충전소마다 이용 편차가 큰 이유는 전기차 보급 목표에 따라 구역별로  충전소를 나눠 설치했기 때문이라는 지적도 있다.

- 한국전기차충전협동조합은 앞으로 수요가 몰리는 거점을 중심을 중심으로 충전기 설치를 해야하 한다는 의견 주장함.

- 기존의 정부 중심으로 공급자 입장에서 설치하던 충전소를 친환경 자동차 차주를 고려해 이용자 중심으로 변화 설치가 필요하다는 지적도 있다.

즉, **꾸준히 증가하는 친환경 차량의 수요를 위해 저비용 고효율의 충전소 설치를 목표 한다.** 충전소 설치 사업자의 기대 수익 최적화를 위해 사용자 입장을 고려한 체계적이며 최적의 입지 선정 결과를 제시함.

## 사용 데이터

1. 충전소 위치 및 현황 정보
2. 급속충전기 보급 현황
3. LPG충전소 현황
4. 오존 월별 대기오염도
5. 아황산가스 월별 대기오염도
6. 이산화질소 월별 대기오염도
7. 미세먼지 월별 대기오염도
8. 행정구역별 인구수
9. 항만 현황
10. 지역별 전기차 등록대 수

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import osmnx as ox
import networkx as nx
import numpy as np
from scipy.stats import norm
import plotly.express as px
import plotly.graph_objects as go
import pgmpy as pg
from pgmpy.factors.discrete import JointProbabilityDistribution as JPD

## 데이터 적합성 판단 기준

In [125]:
# 환경 요인 판단 기준
o3_standard = 0.003     # 오존
so2_standard = 0.002    # 아황산가스
no2_standard = 0.012    # 이산화질소    
co_standard = 0.3       # 일산화탄소
pm10_standard = 15      # 미세먼지
pm25_standard = 10      # 초미세먼지


# 사회 요인 판단 기준
density_standard = 40_000  # 인구밀도
floating_density_standard = 400_000   # 유동인구밀도

EV_per_charger_standard = 5.1     # 전기차충전기당 전기차수
HV_per_charger_standard = 346.53    # 수소차충전기당 수소차수
street_supply_standard = 0.3     # 도로보급률

# 경제 요인 판단 기준
EVCS_cost_standard = 35_000_000             # 급속 충전기 설치비용
# 17_500_000(정부보조금 최대지원가능금액, 출처 한국에너지공단 '2022년 전기차충전서비스산업육성산업) / 0.5(보조율) = 35_000_000
HVCS_construction_cost_standard = 3_000_000_000    # 수소충전소 구축 비용
# 1_500_000_000(정부보조금 최대지원가능금액, 출처 환경부'2022년 수소차 보급 및 충전소 설치사업 보조금 업무처리지침 / 0.5 (일반수소충전소 보조율) = 3_000_000_000(구축비용)
# 4_200_000_000(정부보조금 최대지원가능금액, 출처 환경부'2022년 수소차 보급 및 충전소 설치사업 보조금 업무처리지침 / 0.7 (특수수소충전소 보조율) = 6_000_000_000(구축비용)
income_standard = 3_600          # 연 평균소득
maintenance_cost_standard = 100      # 유지비용
HVCS_income_standard = 0        # 수소충전소 기대수익

# 기술 요인 판단 기준
maximum_charge_standard = 45.3    # 충전가능 수소차 수

In [12]:
def advanced_replace(df, col, str, regex):
    '''
    DataFrame 특정 열의 특수문자 제거
    :param df: 대상 DataFrame
    :param col: column
    :param str: 제거할 특수 문자.
    :param regex: 적용할 정규표현식
    :return:
    '''
    df[col] = df[col].replace(str,0)
    df[col] = df[col].replace(to_replace=regex, value=r'', regex=True)
    return df

In [13]:
def show_norm(mean,std,min,max):
     '''
     정규분포 함수를 matplotlib을 통해 시각화
     mean: 평균값
     std: 표준편차
     min: 관측된 데이터의 최솟값
     max: 관측된 데이터의 최댓값
     '''
     min = min - std
     # x값 설정(min에서 max까지 unit만큼씩 증가하는 x)
     x = np.linspace(min,max,100)
     # y값은 평균이 mean이고 표준편차가 std인 확률밀도함수
     fig = px.line(x=x,y=norm.pdf(x,loc=mean,scale=std))
     fig.show()

In [14]:
def cal_norm(mean,std,min,max,value,affect):
     '''
     정규분포에서 value에 대한 누적확률 구하기
     mean: 평균값
     std: 표준편차
     min: 관측된 데이터의 최솟값
     max: 관측된 데이터의 최댓값
     value: 누적확률을 구하고자 하는 데이터의 측정값
     affect: 입지에 긍정/부정적인 영향(True/False)
     '''
     min = min - std
     red ='rgb(239,85,59)'
     blue = 'rgb(100,110,250)'
     fig = go.Figure()
     
     # 확률 값을 구할 특정 구간의 범위 설정
     cum_a = np.linspace(min,value,100)
     cum_b = np.linspace(value,max,100)
     
     # value까지의 누적확률에서 min까지의 누적확률을 뺌
     pro = norm(mean,std).cdf(value)-norm(mean,std).cdf(min).round(3)
     if affect == True:
          # 구간 사이에 색을 채움
          fig.add_trace(go.Scatter(x=cum_a,y=norm.pdf(cum_a,mean,std), fill = 'tozeroy',name='적합',line=dict(color=blue)))
          fig.add_trace(go.Scatter(x=cum_b,y=norm.pdf(cum_b,mean,std), fill = 'tozeroy',name='부적합',line=dict(color=red)))
          fig.update_yaxes(visible=False)
          annotations = []
          annotations.append(dict(x=value, y=norm.pdf(value,loc=mean,scale=std), showarrow=False, text=round(pro,3), font=dict(size=15,color=blue), xshift=-40, yshift=-100, bordercolor=blue,borderwidth=2))
          annotations.append(dict(x=value, y=norm.pdf(value,loc=mean,scale=std), showarrow=False, text=round(1-pro,3),font=dict(size=15,color=red),xshift=40, yshift=-100, bordercolor=red,borderwidth=2))
     else:
          fig.add_trace(go.Scatter(x=cum_a,y=norm.pdf(cum_a,mean,std), fill = 'tozeroy',name='부적합',line=dict(color=red)))
          fig.add_trace(go.Scatter(x=cum_b,y=norm.pdf(cum_b,mean,std), fill = 'tozeroy',name='적합',line=dict(color=blue)))
          fig.update_yaxes(visible=False)
          annotations = []
          annotations.append(dict(x=value, y=norm.pdf(value,loc=mean,scale=std), showarrow=False, text=round(pro,3), font=dict(size=15,color=red), xshift=-40, yshift=-100, bordercolor=red,borderwidth=2))
          annotations.append(dict(x=value, y=norm.pdf(value,loc=mean,scale=std), showarrow=False, text=round(1-pro,3),font=dict(size=15,color=blue),xshift=40, yshift=-100, bordercolor=blue,borderwidth=2))     
          pro = 1 - pro
     fig.update_layout(annotations=annotations)
     
     fig.show()

     # 최종 누적확률 반환
     return pro, 1-pro

In [15]:
# 데이터의 수를 통한 확률 계산
def cal_pro(dataframe,column,standard,affect):
    if dataframe[column].dtypes == 'dtype(\'O\')':
        num = dataframe[dataframe[column]==standard].shape[0]
        total_num = dataframe.shape[0]
    else:
        if affect == True:
            num = dataframe[dataframe[column]>=standard].shape[0]
            total_num = dataframe.shape[0]
        else:
            num = dataframe[dataframe[column]<standard].shape[0]
            total_num = dataframe.shape[0]

    probability = num/total_num

    return probability, 1-probability

## 상위 요소 - 환경적 요인

대기오염도를 기반하여 확률 값을 계산한다.

### 대기오염도 데이터 불러오기

In [16]:
air_pollution_file_path = "C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\대기오염시도별실시간측정정보(환경부).csv"
df_air_pollution = pd.read_csv(air_pollution_file_path, encoding='cp949')
df_air_pollution

,sidoName,stationName,mangName,so2Value,coValue,o3Value,no2Value,pm10Value,pm25Value,khaiValue
0,충북,송정동(봉명동),도시대기,0.002,0.6,0.003,0.022,65.0,12.0,56.0
1,충북,사천동,도시대기,0.002,0.5,0.002,0.019,28.0,22.0,59.0
2,충북,용담동,도시대기,NaN,0.4,0.002,0.018,12.0,11.0,30.0
3,충북,용암동,도시대기,0.002,0.5,0.002,0.017,31.0,15.0,52.0
4,충북,복대동,도로변대기,0.004,0.7,0.002,0.028,25.0,10.0,47.0
...,...,...,...,...,...,...,...,...,...,...
10907,강원,묵호항,항만,0.013,1.3,0.003,0.034,33.0,16.0,56.0
10908,강원,철원(DMZ),교외대기,0.001,0.3,0.023,0.010,28.0,27.0,82.0
10909,강원,화천(DMZ),교외대기,0.001,0.3,0.033,0.007,16.0,15.0,64.0
10910,강원,인제(DMZ),교외대기,0.001,0.2,0.038,0.005,10.0,8.0,57.0


### 결측값 보간

In [17]:
df_air_pollution = df_air_pollution.interpolate(method='pad')
df_air_pollution

,sidoName,stationName,mangName,so2Value,coValue,o3Value,no2Value,pm10Value,pm25Value,khaiValue
0,충북,송정동(봉명동),도시대기,0.002,0.6,0.003,0.022,65.0,12.0,56.0
1,충북,사천동,도시대기,0.002,0.5,0.002,0.019,28.0,22.0,59.0
2,충북,용담동,도시대기,0.002,0.4,0.002,0.018,12.0,11.0,30.0
3,충북,용암동,도시대기,0.002,0.5,0.002,0.017,31.0,15.0,52.0
4,충북,복대동,도로변대기,0.004,0.7,0.002,0.028,25.0,10.0,47.0
...,...,...,...,...,...,...,...,...,...,...
10907,강원,묵호항,항만,0.013,1.3,0.003,0.034,33.0,16.0,56.0
10908,강원,철원(DMZ),교외대기,0.001,0.3,0.023,0.010,28.0,27.0,82.0
10909,강원,화천(DMZ),교외대기,0.001,0.3,0.033,0.007,16.0,15.0,64.0
10910,강원,인제(DMZ),교외대기,0.001,0.2,0.038,0.005,10.0,8.0,57.0


### 부산 대기오염 데이터 추출

In [18]:
df_air_pollution = df_air_pollution[df_air_pollution['sidoName']=='부산']
df_air_pollution.reset_index(drop=True,inplace=True)
df_air_pollution

,sidoName,stationName,mangName,so2Value,coValue,o3Value,no2Value,pm10Value,pm25Value,khaiValue
0,부산,광복동,도시대기,0.002,0.3,0.016,0.018,11.0,8.0,30.0
1,부산,초량동,도로변대기,0.002,0.4,0.010,0.028,16.0,5.0,47.0
2,부산,태종대,도시대기,0.003,0.2,0.019,0.020,15.0,10.0,33.0
3,부산,청학동,도시대기,0.003,0.2,0.021,0.018,10.0,7.0,35.0
4,부산,전포동,도시대기,0.002,0.2,0.016,0.014,14.0,10.0,37.0
...,...,...,...,...,...,...,...,...,...,...
625,부산,하단동,도시대기,0.003,0.4,0.015,0.029,43.0,23.0,66.0
626,부산,하단동,도시대기,0.003,0.3,0.023,0.015,31.0,22.0,59.0
627,부산,하단동,도시대기,0.002,0.5,0.005,0.031,25.0,17.0,51.0
628,부산,하단동,도시대기,0.004,0.5,0.002,0.047,27.0,17.0,78.0


#### 부산시 오존 적합도

In [19]:
o3_pro,o3_pro_negative = cal_norm(df_air_pollution['o3Value'].mean(),df_air_pollution['o3Value'].std(),
df_air_pollution['o3Value'].min(),df_air_pollution['o3Value'].max(),o3_standard,False)
o3_pro

0.8017202428389887

#### 부산시 아황산가스 적합도

In [20]:
so2_pro,so2_pro_negative = cal_norm(df_air_pollution['so2Value'].mean(),
                                    df_air_pollution['so2Value'].std(),
                                    df_air_pollution['so2Value'].min(),
                                    df_air_pollution['so2Value'].max(),
                                    so2_standard,
                                    False)
so2_pro

0.7911203783441816

#### 부산시 이산화질소 적합도

In [21]:
no2_pro,no2_pro_negative = cal_norm(df_air_pollution['no2Value'].mean(),df_air_pollution['no2Value'].std(),
df_air_pollution['no2Value'].min(),df_air_pollution['no2Value'].max(),no2_standard,False)
no2_pro

0.909590433996196

#### 부산시 일산화탄소 적합도

In [22]:
co_pro,co_pro_negative = cal_norm(df_air_pollution['coValue'].mean(),df_air_pollution['coValue'].std(),
df_air_pollution['coValue'].min(),df_air_pollution['coValue'].max(),co_standard,False)
co_pro

0.8287531282163694

#### 부산시 미세먼지 적합도

In [23]:
pm10_pro,pm10_pro_negative = cal_norm(df_air_pollution['pm10Value'].mean(),df_air_pollution['pm10Value'].std(),
df_air_pollution['pm10Value'].min(),df_air_pollution['pm10Value'].max(),pm10_standard,False)
pm10_pro

0.8866811666335976

In [24]:
pm25_pro,pm25_pro_negative = cal_norm(df_air_pollution['pm25Value'].mean(),df_air_pollution['pm25Value'].std(),
df_air_pollution['pm25Value'].min(),df_air_pollution['pm25Value'].max(),pm25_standard,False)
pm25_pro

0.8415688322677302

## 상위 요소 - 사회적 요인

### 인구밀도 

In [25]:
people_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\행정구역 읍면동 주민등록인구.csv'
df_density = pd.read_csv(people_file_path, encoding='cp949')
df_density = df_density.sort_values(by=["행정구역(시군구)별","행정구역(동읍면)별"],ascending=True).reset_index(drop=True)
df_density

,행정구역(시군구)별,행정구역(동읍면)별,총인구수 (명),면적(km^2),인구밀도(km^2)
0,강서구,가덕도동,4081,19.42,210.0
1,강서구,가락동,2312,14.09,164.0
2,강서구,강동동,4179,16.31,256.0
3,강서구,녹산동,35872,58.93,609.0
4,강서구,대저1동,6417,13.66,470.0
...,...,...,...,...,...
200,해운대구,좌2동,31635,0.50,63270.0
201,해운대구,좌3동,15891,0.72,22071.0
202,해운대구,좌4동,22883,1.80,12713.0
203,해운대구,중1동,27840,1.73,16092.0


In [26]:
23107-16893

6214

In [27]:
density_pro, density_pro_negative = cal_norm(df_density['인구밀도(km^2)'].mean(),
                                             df_density['인구밀도(km^2)'].std(),
                                             df_density['인구밀도(km^2)'].min(),
                                             df_density['인구밀도(km^2)'].max(),
                                             density_standard,
                                             False)

### 유동인구밀도

In [28]:
foottraffic_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산 유동인구 데이터.csv'
df_foottraffic = pd.read_csv(foottraffic_file_path, encoding='cp949')
df_foottraffic

,구군,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월,월 평균
0,강서구,"325,284.39","323,049.87","335,120.06","340,570.52","332,289.77","340,678.45","337,488.64","328,374.74","328,764.38","333,414.26","340,405.10","337,753.15","333,599.44"
1,금정구,"482,598.89","477,047.11","484,669.59","490,864.46","487,435.45","484,936.00","483,898.58","482,964.23","487,082.06","484,633.90","485,502.59","488,392.99","485,002.15"
2,기장군,"341,181.17","343,398.79","342,489.52","345,184.20","348,793.68","348,500.32","350,763.40","349,328.59","348,077.97","348,449.77","345,905.89","344,245.29","346,359.88"
3,남구,"539,072.58","526,693.91","524,611.67","524,961.53","526,758.54","531,078.89","529,810.42","530,199.16","531,658.35","525,870.37","528,106.40","531,610.18","529,202.67"
4,동구,"185,466.33","185,459.30","189,082.61","192,702.25","189,819.14","191,618.73","189,864.16","189,368.74","190,328.67","191,182.03","194,732.77","193,341.86","190,247.22"
5,동래구,"553,655.76","549,718.60","543,091.96","535,439.53","533,908.61","528,124.07","527,730.64","525,468.79","532,656.70","527,498.20","520,126.56","524,788.88","533,517.36"
6,부산진구,"757,261.10","760,742.04","760,212.63","761,728.33","765,460.09","764,636.01","759,638.81","754,837.78","763,942.81","761,731.75","763,818.34","768,957.14","761,913.90"
7,북구,"559,133.62","554,946.17","543,084.57","543,479.25","543,145.91","540,381.96","542,289.98","546,905.11","551,394.74","542,887.88","539,732.97","545,952.52","546,111.22"
8,사상구,"496,102.34","492,871.78","493,054.71","496,932.63","494,094.59","494,207.66","492,973.61","490,296.85","493,842.47","491,808.21","493,815.86","495,297.44","493,774.85"
9,사하구,"641,541.35","632,045.91","628,461.73","633,805.98","630,333.83","629,635.14","630,260.69","631,377.03","633,402.47","627,551.97","625,380.25","631,475.21","631,272.63"


In [29]:
foottraffic_col = df_foottraffic.iloc[:,1:-1].columns.tolist()

In [30]:
df_foottraffic.drop(columns=foottraffic_col, inplace=True)
df_foottraffic.set_index('구군',inplace=True)
df_foottraffic

,월 평균
구군,
강서구,"333,599.44"
금정구,"485,002.15"
기장군,"346,359.88"
남구,"529,202.67"
동구,"190,247.22"
동래구,"533,517.36"
부산진구,"761,913.90"
북구,"546,111.22"
사상구,"493,774.85"


In [31]:
advanced_replace(df_foottraffic, '월 평균','-',r'[^0-9.0-9]')
df_foottraffic['월 평균'] = df_foottraffic['월 평균'].astype(float)

In [32]:
density = []
population = df_foottraffic['월 평균'].tolist()
area = df_density['면적(km^2)'].tolist()
list_p_a = list(zip(population,area))
for x,y in list_p_a:
    val = x/y
    density.append(val)

In [33]:
df_foottraffic.insert(1,'유동인구밀도(km2당)',density)
df_foottraffic

,월 평균,유동인구밀도(km2당)
구군,,
강서구,333599.44,1.717814e+04
금정구,485002.15,3.442173e+04
기장군,346359.88,2.123604e+04
남구,529202.67,8.980191e+03
동구,190247.22,1.392732e+04
동래구,533517.36,1.967973e+04
부산진구,761913.90,6.144467e+04
북구,546111.22,1.680342e+05
사상구,493774.85,4.614718e+05


In [34]:
floating_pro,floating_pro_negative = cal_norm(df_foottraffic['월 평균'].mean(),
                                              df_foottraffic['월 평균'].std(),
                                              df_foottraffic['월 평균'].min(),          
                                              df_foottraffic['월 평균'].max(),  
                                              floating_density_standard,
                                              False)
floating_pro

0.5848617448890694

### 친환경차 등록대수

In [35]:
vehicle_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\연료별_자동차등록대수.csv'
df_vehicle = pd.read_csv(vehicle_file_path, encoding='cp949')
df_vehicle

,시도,시군구,읍면동,수소,전기
0,부산광역시,강서구,강동동,6,71
1,부산광역시,강서구,구랑동,2,35
2,부산광역시,강서구,녹산동,2,32
3,부산광역시,강서구,눌차동,0,15
4,부산광역시,강서구,대저1동,4,290
...,...,...,...,...,...
132,부산광역시,해운대구,송정동,2,176
133,부산광역시,해운대구,우동,35,5091
134,부산광역시,해운대구,재송동,25,1895
135,부산광역시,해운대구,좌동,32,1902


### LPG 충전소 현황 데이터 불러오기 (수소차)

In [36]:
LPG_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산 LPG 충전소 현황(한국가스안전공사).csv'
df_LPG = pd.read_csv(LPG_file_path, encoding='cp949')
df_LPG

,행정시,행정구,업소명,소재지,관리구분,위도,경도,면적,공시지가,토지비용,교차로 수
0,부산,강서구,개인택시신공항충전소,부산 강서구 경전철로188번길 95,자동차충전,35.198669,128.966594,2545.0,2144000.0,5.456480e+09,52
1,부산,강서구,부산공항LPG충전소,부산 강서구 공항로 459,자동차충전,35.149064,128.951891,3105.0,1174000.0,3.645270e+09,16
2,부산,강서구,신항만에너지,부산 강서구 낙동남로 507,자동차충전,35.112947,128.875260,3105.0,1174000.0,3.645270e+09,76
3,부산,강서구,(주)지원네트웍스 부산강서LPG충전소,부산 강서구 낙동북로 310,자동차충전,35.212507,128.962572,1396.0,1140000.0,1.591440e+09,32
4,부산,강서구,신명지에너지,부산 강서구 르노삼성대로 560,자동차충전,35.097554,128.927351,481.0,700300.0,3.368443e+08,159
...,...,...,...,...,...,...,...,...,...,...,...
58,부산,연제구,부산개인택시엘피지충전소,부산 연제구 월드컵대로 399,자동차충전,35.186013,129.055974,9212.0,688600.0,6.343383e+09,146
59,부산,영도구,(주)태종에너지,부산 영도구 영선대로 41,자동차충전,35.084996,129.040033,1610.2,2679000.0,4.313726e+09,255
60,부산,영도구,부산에너지,부산 영도구 해양로 160,용기+자동차+탱크+13kg용기,35.091275,129.069484,5801.0,989400.0,5.739509e+09,67
61,부산,영도구,개인택시 영도LPG충전소,부산 영도구 해양로 296,자동차충전,35.082176,129.076429,1818.0,349800.0,6.359364e+08,98


In [37]:
# 복합충전소를 위해 1500 m^2이상의 부지가 요구됨
df_LPG = df_LPG[df_LPG['면적'] >= 1500].reset_index(drop=True)
df_LPG

,행정시,행정구,업소명,소재지,관리구분,위도,경도,면적,공시지가,토지비용,교차로 수
0,부산,강서구,개인택시신공항충전소,부산 강서구 경전철로188번길 95,자동차충전,35.198669,128.966594,2545.0,2144000.0,5.456480e+09,52
1,부산,강서구,부산공항LPG충전소,부산 강서구 공항로 459,자동차충전,35.149064,128.951891,3105.0,1174000.0,3.645270e+09,16
2,부산,강서구,신항만에너지,부산 강서구 낙동남로 507,자동차충전,35.112947,128.875260,3105.0,1174000.0,3.645270e+09,76
3,부산,강서구,르노삼성자동차(주),부산 강서구 르노삼성대로 61,자동차충전,35.095408,128.883215,1538.0,640100.0,9.844738e+08,82
4,부산,강서구,죽림LPG충전소,부산 강서구 호계로 102,자동차충전,35.200736,128.893462,1580.0,1017000.0,1.606860e+09,16
5,부산,강서구,가락충전소,부산 강서구 호계로 195,자동차충전,35.208847,128.890148,2600.0,1017000.0,2.644200e+09,20
6,부산,금정구,(주)해연,부산 금정구 개좌로 225-15,용기+탱크+13kg용기,35.230796,129.122388,3346.0,1370000.0,4.584020e+09,119
7,부산,금정구,(주)신진가스,부산 금정구 반송로 433,자동차충전,35.215006,129.111143,1646.4,2797000.0,4.604981e+09,407
8,부산,기장군,마라톤가스(주)청강충전소,부산 기장군 기장읍 기장대로 343 A동,자동차충전,35.225657,129.213988,2592.0,1069000.0,2.770848e+09,45
9,부산,기장군,한국에너지산업(주),부산 기장군 기장읍 기장대로413번길 38,용기+자동차+탱크+13kg용기,35.235543,129.215629,2473.0,1063000.0,2.628799e+09,241


In [38]:
LPG_group = df_LPG.groupby('행정구').count()
LPG_group

,행정시,업소명,소재지,관리구분,위도,경도,면적,공시지가,토지비용,교차로 수
행정구,,,,,,,,,,
강서구,6,6,6,6,6,6,6,6,6,6
금정구,2,2,2,2,2,2,2,2,2,2
기장군,8,8,8,8,8,8,8,8,8,8
동구,2,2,2,2,2,2,2,2,2,2
북구,2,2,2,2,2,2,2,2,2,2
사상구,3,3,3,3,3,3,3,3,3,3
사하구,3,3,3,3,3,3,3,3,3,3
연제구,1,1,1,1,1,1,1,1,1,1
영도구,3,3,3,3,3,3,3,3,3,3


### 전기차 충전기수

In [39]:
EVCS_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\전기차충전소 리스트.csv'
df_EVCS = pd.read_csv(EVCS_file_path, encoding='cp949')
df_EVCS

,운영기관,충전소,충전기ID,충전기타입,지역,시군구,행정동,주소,이용가능시간,이용자 제한,충전용량,비고
0,환경부(한국자동차환경협회),kbs 부산방송총국,1,DC차데모+AC3상+DC콤보,부산광역시,수영구,남천동,부산광역시 수영구 수영로 429,24시간 이용가능,NaN,200kW멀티,NaN
1,환경부(한국자동차환경협회),kbs 부산방송총국,11,DC콤보,부산광역시,수영구,남천동,부산광역시 수영구 수영로 429,24시간 이용가능,NaN,200kW멀티,NaN
2,환경부(한국자동차환경협회),kbs 부산방송총국,12,DC콤보,부산광역시,수영구,남천동,부산광역시 수영구 수영로 429,24시간 이용가능,NaN,200kW멀티,NaN
3,환경부(한국자동차환경협회),국립부산과학관,11,DC콤보,부산광역시,기장군,기장읍,부산광역시 기장군 기장읍 석산리 293,24시간 이용가능,NaN,100kW멀티,NaN
4,환경부(한국자동차환경협회),국립부산과학관,12,DC콤보,부산광역시,기장군,기장읍,부산광역시 기장군 기장읍 석산리 293,24시간 이용가능,NaN,100kW멀티,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6602,한국전자금융,부산 플러스타워,1,AC완속,부산광역시,강서구,명지동,부산광역시 강서구 명지국제1로 56-1플러스타워(3598-2 플러스타워),24시간 이용가능,NaN,7kW,NaN
6603,한국전자금융,부산 플러스타워,2,AC완속,부산광역시,강서구,명지동,부산광역시 강서구 명지국제1로 56-1플러스타워(3598-2 플러스타워),24시간 이용가능,NaN,7kW,NaN
6604,한국전자금융,부산 휴마트괴정점,1,AC완속,부산광역시,사하구,괴정동,부산광역시 사하구 낙동대로 123휴마트괴정점(괴정동 358-1),24시간 이용가능,NaN,7kW,NaN
6605,한국전자금융,부산 휴마트괴정점,2,AC완속,부산광역시,사하구,괴정동,부산광역시 사하구 낙동대로 123휴마트괴정점(괴정동 358-1),24시간 이용가능,NaN,7kW,NaN


In [40]:
# si = []
# gu = []
# for i in range(df_EVCS.shape[0]):
#     strings = df_EVCS.iloc[i]['addr'].split()
#     si.append(strings[0])
#     gu.append(strings[1])
# df_EVCS.drop(columns='addr',inplace=True)
# df_EVCS.insert(4,'addr_gu',gu)
# df_EVCS.insert(4,'addr_si',si)
# df_EVCS.to_csv(EVCS_file_path,encoding='cp949',index=False)

#### 급속 완속 여부
    완속충전타입:AC완속 
    급속충전타입:DC차데모, AC3상, DC콤보
    chger_type
    1:DC차데모
    2:AC완속
    3:DC차데모+AC3상
    4:DC콤보
    5:DC차데모+DC콤보
    6:DC차데모+AC3상+DC콤보

In [41]:
df_EVCS_standard = df_EVCS[df_EVCS['충전기타입']=='AC완속'].reset_index(drop=True)
df_EVCS_fast = df_EVCS[df_EVCS['충전기타입']!='AC완속'].reset_index(drop=True)
df_EVCS_fast

,운영기관,충전소,충전기ID,충전기타입,지역,시군구,행정동,주소,이용가능시간,이용자 제한,충전용량,비고
0,환경부(한국자동차환경협회),kbs 부산방송총국,1,DC차데모+AC3상+DC콤보,부산광역시,수영구,남천동,부산광역시 수영구 수영로 429,24시간 이용가능,NaN,200kW멀티,NaN
1,환경부(한국자동차환경협회),kbs 부산방송총국,11,DC콤보,부산광역시,수영구,남천동,부산광역시 수영구 수영로 429,24시간 이용가능,NaN,200kW멀티,NaN
2,환경부(한국자동차환경협회),kbs 부산방송총국,12,DC콤보,부산광역시,수영구,남천동,부산광역시 수영구 수영로 429,24시간 이용가능,NaN,200kW멀티,NaN
3,환경부(한국자동차환경협회),국립부산과학관,11,DC콤보,부산광역시,기장군,기장읍,부산광역시 기장군 기장읍 석산리 293,24시간 이용가능,NaN,100kW멀티,NaN
4,환경부(한국자동차환경협회),국립부산과학관,12,DC콤보,부산광역시,기장군,기장읍,부산광역시 기장군 기장읍 석산리 293,24시간 이용가능,NaN,100kW멀티,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
492,모던텍,유엔공원 EV충전소(맘차지),1,DC차데모+DC콤보,부산광역시,남구,대연동,부산광역시 남구 석포로 134SK 유엔주유소 옆(대연동 949-4),08:00 ~ 22:00,NaN,50kW단독,NaN
493,모던텍,유엔공원 EV충전소(맘차지),2,DC차데모+DC콤보,부산광역시,남구,대연동,부산광역시 남구 석포로 134SK 유엔주유소 옆(대연동 949-4),08:00 ~ 22:00,NaN,50kW단독,NaN
494,모던텍,유엔공원 EV충전소(맘차지),3,DC차데모+DC콤보,부산광역시,남구,대연동,부산광역시 남구 석포로 134SK 유엔주유소 옆(대연동 949-4),08:00 ~ 22:00,NaN,50kW단독,NaN
495,모던텍,유엔공원 EV충전소(맘차지),4,DC차데모+DC콤보,부산광역시,남구,대연동,부산광역시 남구 석포로 134SK 유엔주유소 옆(대연동 949-4),08:00 ~ 22:00,NaN,50kW단독,NaN


In [42]:
# 구별 전기차 완속충전기 수
EVCS_standard_group = df_EVCS_standard.groupby(by=['시군구','행정동'])

# 구별 전기차 급속충전기 수
EVCS_fast_group = df_EVCS_fast.groupby(by=['시군구','행정동'])

### 전기차 충전기당 전기차 수

   충전기 1기당 전기차 수 = 행정구 전기차 수 / 행정구 충전기 수  

In [43]:
EV_per_charger = []
dong_vehicle = df_vehicle['읍면동'].tolist()
dong_EVCS = df_EVCS['행정동'].tolist()
dong_EVCS = list(set(dong_EVCS))
for dong in dong_vehicle:
    if dong in dong_EVCS:           
        result = df_vehicle[df_vehicle['읍면동']==dong]['전기'].values[0]  / df_EVCS[df_EVCS['행정동']==dong]['충전기ID'].count()
    else:
        result = df_vehicle[df_vehicle['읍면동']==dong]['전기'].values[0]
    EV_per_charger.append(result)
df_vehicle.insert(5,'전기차충전소당 전기차 수', EV_per_charger)

In [44]:
df_vehicle

,시도,시군구,읍면동,수소,전기,전기차충전소당 전기차 수
0,부산광역시,강서구,강동동,6,71,14.200000
1,부산광역시,강서구,구랑동,2,35,17.500000
2,부산광역시,강서구,녹산동,2,32,32.000000
3,부산광역시,강서구,눌차동,0,15,15.000000
4,부산광역시,강서구,대저1동,4,290,24.166667
...,...,...,...,...,...,...
132,부산광역시,해운대구,송정동,2,176,3.914894
133,부산광역시,해운대구,우동,35,5091,10.695378
134,부산광역시,해운대구,재송동,25,1895,21.292135
135,부산광역시,해운대구,좌동,32,1902,6.558621


In [45]:
EV_per_charger_pro, EV_per_charger_pro_negative = cal_norm(df_vehicle['전기차충전소당 전기차 수'].mean(),
                                                           df_vehicle['전기차충전소당 전기차 수'].std(),
                                                           df_vehicle['전기차충전소당 전기차 수'].min(),
                                                           df_vehicle['전기차충전소당 전기차 수'].max(),
                                                           EV_per_charger_standard,
                                                           False)

### 전기차 충전소 이용률 계산

In [46]:
userate_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\실시간 전기차 충전소 상태정보.csv'
df_userate = pd.read_csv(userate_file_path,encoding='cp949')
df_userate.sort_values(by=['region_2depth_name','region_3depth_name'])

,statNm,statId,chgerId,chgerType,addr,region_2depth_name,region_3depth_name,lat,lng,useTime,...,lastTsdt,lastTedt,nowTsdt,output,method,zcode,zscode,kind,kindDetail,parkingFree
74,대저역 (대저차량기지),ME20C190,91,4,부산광역시 강서구 대저1동 1619,강서구,대저1동,35.213220,128.954243,24시간 이용가능,...,2.020000e+13,2.020000e+13,NaN,100,단독,26,26440.0,A0,A003,Y
118,(주)응카,KE000593,1,4,부산광역시 강서구 대저중앙로394번길 32,강서구,대저1동,35.219401,128.988252,"주중 08:00~19:00, 주말 09:00~18:00",...,2.020000e+13,2.020000e+13,2.020000e+13,100,동시,26,26440.0,H0,H003,Y
119,(주)응카,KE000593,2,4,부산광역시 강서구 대저중앙로394번길 32,강서구,대저1동,35.219401,128.988252,"주중 08:00~19:00, 주말 09:00~18:00",...,2.020000e+13,2.020000e+13,2.020000e+13,100,동시,26,26440.0,H0,H003,Y
266,대저역 (대저차량기지),ME20C190,91,4,부산광역시 강서구 대저1동 1619,강서구,대저1동,35.213220,128.954243,24시간 이용가능,...,2.020000e+13,2.020000e+13,NaN,100,단독,26,26440.0,A0,A003,Y
311,(주)응카,KE000593,1,4,부산광역시 강서구 대저중앙로394번길 32,강서구,대저1동,35.219401,128.988252,"주중 08:00~19:00, 주말 09:00~18:00",...,2.020000e+13,2.020000e+13,2.020000e+13,100,동시,26,26440.0,H0,H003,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6791,부산달맞이주유소,GS000075,2,4,부산광역시 해운대구 달맞이길 57,해운대구,중동,35.161533,129.171133,24시간 이용가능,...,2.020000e+13,2.020000e+13,2.020000e+13,200,단독,26,26350.0,E0,E007,Y
6890,중동역 공영주차장,ME174114,1,6,부산광역시 해운대구 해운대로 731,해운대구,중동,35.166826,129.169979,24시간 이용가능,...,2.020000e+13,2.020000e+13,2.020000e+13,50,단독,26,26350.0,B0,B001,N
6907,이마트 해운대점,ME184074,1,6,부산광역시 해운대구 좌동순환로 511,해운대구,중동,35.165994,129.167388,10:00~24:00,...,2.020000e+13,2.020000e+13,NaN,50,단독,26,26350.0,E0,E001,Y
6980,부산달맞이주유소,GS000075,1,4,부산광역시 해운대구 달맞이길 57,해운대구,중동,35.161533,129.171133,24시간 이용가능,...,2.020000e+13,2.020000e+13,2.020000e+13,200,단독,26,26350.0,E0,E007,Y


In [110]:
# stat : 하루 이용자 수
use_rate = df_userate.groupby('statId').sum()
high_rate = use_rate[use_rate['stat']>=5.1].index.tolist()
use_rate[use_rate['stat']>=EV_per_charger_standard]

,chgerId,chgerType,lat,lng,stat,statUpdDt,lastTsdt,lastTedt,nowTsdt,output,zcode,zscode
statId,,,,,,,,,,,,
EP200480,37,222,1300.406436,4773.259633,8,7.474000e+14,7.474000e+14,7.474000e+14,7.474000e+14,1850,962,981610.0
GN000986,266,190,1342.815466,4908.255055,6,7.676000e+14,7.676000e+14,7.676000e+14,7.676000e+14,1900,988,1014980.0
GS000075,114,304,2672.276508,9817.006108,22,1.535200e+15,1.535200e+15,1.535200e+15,1.535200e+15,15200,1976,2002600.0
HE001095,38,190,1333.227872,4900.908906,12,7.676000e+14,7.676000e+14,7.676000e+14,7.676000e+14,3800,988,1002440.0
HE001097,38,228,1333.844080,4901.232400,13,7.676000e+14,7.676000e+14,7.676000e+14,7.676000e+14,1900,988,1002440.0
...,...,...,...,...,...,...,...,...,...,...,...,...
ST265308,380,608,5348.080161,19604.961110,9,3.070400e+15,3.070400e+15,3.070400e+15,3.070400e+15,15200,3952,4032560.0
ST265309,380,608,5348.080161,19604.961110,9,3.070400e+15,3.070400e+15,3.070400e+15,3.070400e+15,15200,3952,4032560.0
ST267115,38,228,1337.230826,4909.990302,6,7.676000e+14,7.676000e+14,7.676000e+14,7.676000e+14,1900,988,1014980.0


In [48]:
high_rate_dong = []
for name in high_rate:
    dong = df_userate[df_userate['statId']==name]['region_3depth_name'].values[0]
    high_rate_dong.append(dong)
high_rate_dong = list(set(high_rate_dong))

In [49]:
use_rate['stat'].mean()

7.30327868852459

#### 부산 전기차 충전소 적합확률

In [50]:
pro,pro_negative = cal_norm(use_rate['stat'].mean(),
                            use_rate['stat'].std(),
                            use_rate['stat'].min(),
                            use_rate['stat'].max(),
                            EV_per_charger_standard,
                            False)

### 수소충전소 수

In [51]:
HVCS_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산 수소차 충전소 현황.csv'
df_HVCS = pd.read_csv(HVCS_file_path, encoding='cp949')

In [52]:
address_city = []
address_gu = []
for i in range(df_HVCS.shape[0]):
    strings = df_HVCS.iloc[i]['주소'].split()
    address_city.append(strings.pop(0))
    address_gu.append(strings.pop(0))
df_HVCS.insert(2,"시",address_city)
df_HVCS.insert(3,"구",address_gu)    
df_HVCS

,충전소,충전기,시,구,주소,위도,경도,하루평균충전횟수,최대 충전 수소차수,공급방식,충전소 설치방식,압축기용량,충전기용량,충전압력,운영시간,평균가동률,충전가격
0,서부산NK수소충전소,1,부산광역시,강서구,부산광역시 강서구 가락대로 347,35.113171,128.843291,14,67,튜브트레일러,단독,302.63,33.4,700,10,35.0,8800
1,H부산수소충전소,3,부산광역시,사상구,부산광역시 사상구 학감대로 167,35.145864,128.987651,60,70,천연가스개질,복합,308.00,25.0,700,12,41.6,8800
2,하이넷부산정관수소충,1,부산광역시,기장군,부산광역시 기장군 정관읍 신정관로 187,35.328714,129.201710,16,60,천연가스개질,복합,NaN,25.0,700,12,33.3,8800


### 수소차충전기당 수소자동차 수 

In [138]:
# 부산시에 충전소는 3곳으로, 구단위로 구하는 것이 불가능
# 부산시의 수소충전소당 수소자동차 수
HV_per_charger = df_vehicle['수소'].sum() / df_HVCS['충전소'].count()
HV_per_charger

513.0

In [131]:
#### 수소차 충전기당 1일 충전 수요
# 수소차 1일 평균 충전수요
need_HVCS = df_vehicle['수소'].sum() * (1/9.2) 
# 수소차 충전기당 1일 평균 충전수요
need_HVCS_per_charger = need_HVCS / df_HVCS['충전기'].sum()
need_HVCS_per_charger

33.45652173913044

#### 가동률이 50%가 되도록 하는 수소 충전소당 수소차 수
충전소요시간 15분, 가동률 50% => 시간당 2대 충전

충전소 평균 운영시간 11.3h

충전주기 9.2일

충전기 수

충전소 수

2 * 11.3 * 9.2 * 5 / 3 = 346.533

In [55]:
if HV_per_charger >= HV_per_charger_standard:
    HV_per_charger_pro = 1.0
    HV_per_charger_pro_negative = 0
else:
    HV_per_charger_pro = 0
    HV_per_charger_pro_negative = 1.0

### 도로 보급률

In [56]:
# 주차장 정보
street_supply_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\도로보급률.csv'
df_street_supply = pd.read_csv(street_supply_file_path, encoding='cp949')
df_street_supply

,행정구,행정동,도로연장(km),면적(km^2),도로보급률,도로보급률 지수
0,강서구,가덕도동,63.024042,19.42,7.079431,0.397613
1,강서구,가락동,50.605329,14.09,8.866390,0.776725
2,강서구,강동동,28.907008,16.31,3.501386,0.212054
3,강서구,녹산동,305.938534,58.93,6.654082,0.072362
4,강서구,대저1동,86.238678,13.66,9.211084,0.491914
...,...,...,...,...,...,...
200,해운대구,좌2동,5.592018,0.50,1.406046,0.176767
201,해운대구,좌3동,7.319492,0.72,2.163909,0.319865
202,해운대구,좌4동,15.660067,1.80,2.440062,0.190098
203,해운대구,중1동,21.117586,1.73,3.042892,0.219229


In [57]:
street_supply_pro, street_supply_pro_negative = cal_norm(df_street_supply['도로보급률 지수'].mean(),
                                                         df_street_supply['도로보급률 지수'].std(),
                                                         df_street_supply['도로보급률 지수'].min(),
                                                         df_street_supply['도로보급률 지수'].max(),
                                                         street_supply_standard,
                                                         False)

## 상위 요소 - 경제적 요소

### 전기차 충전기 비용

In [58]:
EVCS_cost_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\전기차 충전기 가격.csv'
df_EVCS_cost = pd.read_csv(EVCS_cost_file_path, encoding='cp949')
df_EVCS_cost

,모델명,타입,충전기가격,전격용량,충전기
0,JC-6511PS-B-PO-BC,벽부착형 또는 스탠드형(폴 포함),1800000,7 kW,완속
1,JEV-AW-107,벽부착형,2000000,7 kWh,완속
2,JEV-AS-107,스탠드형,2000000,7 kWh,완속
3,CS 100X 2BC01W,벽부착형,1452080,7 kWh,완속
4,CS 500A 2BC04W,벽부착형,1889248,7 kWh,완속
5,CS 200A 2BC03C,스탠드형,2018074,7 kWh,완속
6,CS 200A 3BC01C,스탠드형,3435449,14 kWh,완속
7,UK-NC7W-TCE,벽부착형 또는 스탠드형(폴 포함),1800000,7 kW,완속
8,PECOH-HSC,벽부착형 또는 스탠드형(폴 포함),770000,7 kW,완속
9,SPVseries,스탠드형,27500000,100kW,급속


In [59]:
EVCS_cost, EVCS_cost_negative = cal_norm(df_EVCS_cost[df_EVCS_cost['충전기']=='급속']['충전기가격'].mean(),
                                         df_EVCS_cost[df_EVCS_cost['충전기']=='급속']['충전기가격'].std(),
                                         df_EVCS_cost[df_EVCS_cost['충전기']=='급속']['충전기가격'].min(),
                                         df_EVCS_cost[df_EVCS_cost['충전기']=='급속']['충전기가격'].max(),
                                         EVCS_cost_standard,
                                         True)

### 수소충전소 구축비용

In [60]:
HVCS_construction_cost_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\수소 충전소 종류별 구축 비용.csv'
df_HVCS_construction_cost = pd.read_csv(HVCS_construction_cost_file_path, encoding='cp949')
df_HVCS_construction_cost = df_HVCS_construction_cost.set_index('충전소 종류')
df_HVCS_construction_cost

,구축 비용
충전소 종류,
파이프라인,2400000000
튜브트레일러,2200000000
이동식,1800000000
천연가스개질,4600000000
수전해,3100000000


#### 수소 충전소 구축 비용 적합도

In [61]:
HVCS_construction_cost_pro,HVCS_construction_cost_pro_negative  = cal_norm(df_HVCS_construction_cost['구축 비용'].mean(),
                                                                           df_HVCS_construction_cost['구축 비용'].std(),
                                                                           df_HVCS_construction_cost['구축 비용'].min(),
                                                                           df_HVCS_construction_cost['구축 비용'].max(),
                                                                           HVCS_construction_cost_standard,
                                                                           True)
HVCS_construction_cost_pro

0.5379381229554602

### 평균소득

In [62]:
income_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산 평균소득.csv'
df_income = pd.read_csv(income_file_path,encoding='cp949')
df_income

,행정구,행정동,가구당인구수,평균소득금액(만)
0,강서구,가덕도동,2.00,2414.330000
1,강서구,가락동,1.77,1765.034000
2,강서구,강동동,1.99,2650.385000
3,강서구,녹산동,1.94,2143.833333
4,강서구,대저1동,1.91,3128.142500
...,...,...,...,...
200,해운대구,좌2동,2.37,3611.697857
201,해운대구,좌3동,2.35,3054.358462
202,해운대구,좌4동,2.49,3271.529231
203,해운대구,중1동,2.69,4010.241000


In [63]:
# gu = []
# dong = []
# for i in range(df_income.shape[0]):
#     text = df_income.iloc[i]['행정동']
#     strings = text.split()
#     gu.append(strings[1])
#     dong.append(strings[2])

In [64]:
# df_income.drop(columns='행정동',inplace=True)
# df_income.insert(0,'행정구',gu)
# df_income.insert(1,'행정동',dong)
# df_income.to_csv(income_file_path,encoding='cp949',index=False)

In [65]:
income_pro, income_pro_negative = cal_norm(df_income['평균소득금액(만)'].mean(),
                                           df_income['평균소득금액(만)'].std(),
                                           df_income['평균소득금액(만)'].min(),
                                           df_income['평균소득금액(만)'].max(),
                                           income_standard,
                                           False)

### 수소차충전소 기대수익

#### 연간 유지비용

연간 운영비는 약 3.25억원이 소요되며, 그 세부내역은 아래와 같다.

- 인건비: 6,000만 원(인건비: 안전관리책임자 1인 4천만 원 + 충전원 1인 2천만 원)
- 전기세 비용: 16,200만 원(칠러: 5 kWh/kg, 압축기: 3.3 kWh/kg 등) [10% 가동 시 1,620만 원, 50% 가동 시 8,100만 원]
- 유지보수비: 3,000만 원(장비 수리비 및 노후장비 교체비용)
- 고압가스 품질검사: 400만 원(100만 원 × 4회)
- 연간 도로점용료: 500만 원(부지 공시지가의5% 가정, 10만 원/m2, 1,000 m2)
- 카드수수료: 5,400만 원(3%, 10,000원/kg) [10% 가동 시 540만 원, 50% 가동 시 2,700만 원]
- 기타: 1,000만 원(공과금, 교육훈련비, 물품구입비, 경비용역비 등)

정부보조금을 제외한 본인 투자금 15억 원, 할인율 10%, 회수기간 15년을 가정하면, 연간수익 2
억 원이 손익분기점이 된다. 수소충전소가 100%가동할 경우에는 판매마진 3,000원/kg으로 유지
가능하며, 초기에 가동률이 10%로 낮을 경우에는판매마진이 3,000원/kg이라면 운영비 보조금을 연
간 2.8억 원 지원하여야 적자를 면할 수 있다

연간 기대수익 = 연간 판매수익 - 연간 유지비용

In [66]:
def cal_maintenance_cost(station_type, operating_rate):
    # 인건비
    if station_type == '복합':
        labor_cost = 40_000_000
    else:
        labor_cost = 60_000_000
    # 전기세
    # 가동률 1%당 162만원
    elec_cost = operating_rate * 1_620_000
    # 카드수수료
    # 가동률 1%당 54만원
    card_cost = operating_rate * 540_000 
    # 기타(장비 교체비용, 품질검사비, 연간 도로점용료, 물품구입비, 공과금 등)
    etc = 30_000_000 + 4_000_000 + 5_000_000 + 10_000_000
    maintenance_cost = labor_cost + elec_cost + card_cost + etc

    return maintenance_cost

#### 연간 판매수익

In [67]:
def cal_revenue(charge_time_average, sale_cost, maintenance_cost):
    # 수소연료 평균 공급단가(1kg당)
    hidrogen_cost = 6_000
    # 수소연료 판매 마진(1kg당)
    revenue_per_kg = sale_cost - hidrogen_cost
    # 수소차 1회 평균 수소연료 충전량(kg)
    charge_amount = 3.736 
    # 일평균 판매수익
    day_revenue = revenue_per_kg * charge_amount * charge_time_average 
    year_revenue = day_revenue * 365
    # 수소연료구입비 보조금
    grant = 0

    # 수소연료구입비 보조금 지원금 반영
    diff = year_revenue - maintenance_cost
    
    if diff < 0:
        diff = -diff
        # 기준단가
        grant_cost_standard = sale_cost - (maintenance_cost / charge_amount * charge_time_average * 365)
        # 지원단가 (수소연료공급단가에 기준단가와 차액의 70%)
        grant_cost = (hidrogen_cost - grant_cost_standard) * 0.7
        # 지원금액
        grant = grant_cost * charge_amount * charge_time_average * 365
        # 최대지원금액은 적자의 80%
        if grant > diff * 0.8:
            grant = diff * 0.8
    # 연간수입 + 보조금
    year_revenue = year_revenue + grant
    # 연간수입에 유지보수비용을 뺀 순수익
    diff = year_revenue - maintenance_cost    

    return diff

#### 연간 기대 수익

In [68]:
revenue = []
for i in range(df_HVCS.shape[0]):
    maintenance_cost = cal_maintenance_cost(df_HVCS.iloc[i]['충전소 설치방식'],df_HVCS.iloc[i]['평균가동률'])
    rev = cal_revenue(df_HVCS.iloc[i]['하루평균충전횟수'], df_HVCS.iloc[i]['충전가격'], maintenance_cost)
    revenue.append(rev)
revenue    

[-26229062.399999976, 50235520.00000003, -19967385.599999994]

In [69]:
df_HVCS.insert(17,'연간기대수익',revenue)
df_HVCS

,충전소,충전기,시,구,주소,위도,경도,하루평균충전횟수,최대 충전 수소차수,공급방식,충전소 설치방식,압축기용량,충전기용량,충전압력,운영시간,평균가동률,충전가격,연간기대수익
0,서부산NK수소충전소,1,부산광역시,강서구,부산광역시 강서구 가락대로 347,35.113171,128.843291,14,67,튜브트레일러,단독,302.63,33.4,700,10,35.0,8800,-26229062.4
1,H부산수소충전소,3,부산광역시,사상구,부산광역시 사상구 학감대로 167,35.145864,128.987651,60,70,천연가스개질,복합,308.00,25.0,700,12,41.6,8800,50235520.0
2,하이넷부산정관수소충,1,부산광역시,기장군,부산광역시 기장군 정관읍 신정관로 187,35.328714,129.201710,16,60,천연가스개질,복합,NaN,25.0,700,12,33.3,8800,-19967385.6


In [70]:
df_HVCS['연간기대수익'].describe()

count    3.000000e+00
mean     1.346357e+06
std      4.245486e+07
min     -2.622906e+07
25%     -2.309822e+07
50%     -1.996739e+07
75%      1.513407e+07
max      5.023552e+07
Name: 연간기대수익, dtype: float64

In [77]:
# 수소충전소 
# 평균 수익: -110_000_000
HVCS_income_pro, HVCS_income_pro_negative = cal_norm(-110_000_000,
                                                     100_000_000,
                                                     -300_000_000,
                                                     160_000_000,
                                                     HVCS_income_standard,
                                                     False)

## 상위 요소 - 기술적 요소

### 전기차 충전기 용량

In [79]:
charge_capacity = df_EVCS_fast.groupby('충전용량')
charge_capacity.describe()

충전기ID                                                       비고       \
         count       mean        std  min   25%   50%   75%   max count mean   
충전용량                                                                           
100kW단독  308.0   2.860390  10.316016  1.0   1.0   1.0   2.0  91.0   0.0  NaN   
100kW멀티   35.0  10.971429  23.319376  1.0   1.0   1.0  11.0  91.0   0.0  NaN   
200kW단독    6.0   1.500000   0.547723  1.0   1.0   1.5   2.0   2.0   0.0  NaN   
200kW멀티   27.0  17.333333   6.120583  1.0  12.0  21.0  22.0  24.0   0.0  NaN   
50kW단독   121.0   1.991736   2.006223  1.0   1.0   1.0   2.0  11.0   0.0  NaN   

                                 
        std min 25% 50% 75% max  
충전용량                             
100kW단독 NaN NaN NaN NaN NaN NaN  
100kW멀티 NaN NaN NaN NaN NaN NaN  
200kW단독 NaN NaN NaN NaN NaN NaN  
200kW멀티 NaN NaN NaN NaN NaN NaN  
50kW단독  NaN NaN NaN NaN NaN NaN

In [99]:
capacity_50only_pro = df_EVCS_fast[df_EVCS_fast['충전용량']=='50kW단독']['운영기관'].count() / df_EVCS_fast['충전용량'].shape[0]
capacity_100only_pro = df_EVCS_fast[df_EVCS_fast['충전용량']=='100kW단독']['운영기관'].count() / df_EVCS_fast['충전용량'].shape[0]
capacity_100multi_pro = df_EVCS_fast[df_EVCS_fast['충전용량']=='100kW멀티']['운영기관'].count() / df_EVCS_fast['충전용량'].shape[0]
capacity_200only_pro = df_EVCS_fast[df_EVCS_fast['충전용량']=='200kW단독']['운영기관'].count() / df_EVCS_fast['충전용량'].shape[0]
capacity_200multi_pro = df_EVCS_fast[df_EVCS_fast['충전용량']=='200kW멀티']['운영기관'].count() / df_EVCS_fast['충전용량'].shape[0]

### 수소차 충전소 공급방식

In [82]:
df_HVCS

,충전소,충전기,시,구,주소,위도,경도,하루평균충전횟수,최대 충전 수소차수,공급방식,충전소 설치방식,압축기용량,충전기용량,충전압력,운영시간,평균가동률,충전가격,연간기대수익
0,서부산NK수소충전소,1,부산광역시,강서구,부산광역시 강서구 가락대로 347,35.113171,128.843291,14,67,튜브트레일러,단독,302.63,33.4,700,10,35.0,8800,-26229062.4
1,H부산수소충전소,3,부산광역시,사상구,부산광역시 사상구 학감대로 167,35.145864,128.987651,60,70,천연가스개질,복합,308.00,25.0,700,12,41.6,8800,50235520.0
2,하이넷부산정관수소충,1,부산광역시,기장군,부산광역시 기장군 정관읍 신정관로 187,35.328714,129.201710,16,60,천연가스개질,복합,NaN,25.0,700,12,33.3,8800,-19967385.6


In [101]:
HVCS_type_tube_pro = df_HVCS[df_HVCS['공급방식']=='튜브트레일러']['충전소'].count() / df_HVCS['충전소'].shape[0]
HVCS_type_reformation_pro = df_HVCS[df_HVCS['공급방식']=='천연가스개질']['충전소'].count() / df_HVCS['충전소'].shape[0]

### 최대 충전 가능한 수소차 수 

수소차 1대 충전시 15분 가량 소요

시간당 충전 가능 차량 수 : 60 / 15 = 4

충전소 평균 운영시간 11.3h

하루 충전가능 수 : 4 * 11.3 = 45.3

In [133]:
maximum_charge_pro, maximum_charge_pro_negative = cal_norm(df_HVCS['최대 충전 수소차수'].mean(),
                                                           df_HVCS['최대 충전 수소차수'].std(),
                                                           df_HVCS['최대 충전 수소차수'].mean()-3*df_HVCS['최대 충전 수소차수'].std(),
                                                           df_HVCS['최대 충전 수소차수'].max()+3*df_HVCS['최대 충전 수소차수'].std(),
                                                           maximum_charge_standard,
                                                           False)

## 결합 확률로 상위 요인 도출하기


In [141]:
## 상위 요인별 하위변수 
# E는 전기차, H는 수소차
variable_environment = [] # 환경변수는 전기/수소 동일
variable_social_E = []
variable_social_H = []
variable_economic_E = []
variable_economic_H = []
variable_technical_E = []
variable_technical_H = []

variable_environment.append(co_pro)
variable_environment.append(so2_pro)
variable_environment.append(no2_pro)
variable_environment.append(o3_pro)
variable_environment.append(pm10_pro)
variable_environment.append(pm25_pro)


variable_social_E.append(density_pro)
variable_social_E.append(floating_pro)
variable_social_E.append(EV_per_charger_pro)
variable_social_E.append(street_supply_pro)

variable_social_H.append(density_pro)
variable_social_H.append(floating_pro)
variable_social_H.append(HV_per_charger_pro)
variable_social_H.append(street_supply_pro)


variable_economic_E.append(EVCS_cost)
variable_economic_E.append(income_pro)

variable_economic_H.append(HVCS_construction_cost_pro)
variable_economic_H.append(income_pro)
variable_economic_H.append(HVCS_income_pro)

capacity_pro = [capacity_50only_pro, capacity_100only_pro, capacity_100multi_pro, capacity_200only_pro, capacity_200multi_pro]
variable_technical_E.append(capacity_pro)

HVCS_type_pro = [HVCS_type_tube_pro, HVCS_type_reformation_pro]
variable_technical_H.append(HVCS_type_pro)
variable_technical_H.append(maximum_charge_pro)


## 상위 요인 결합 가중치
weight_environment = [0.3,0.05,0.3,0.05,0.15,0.15]      # co, so2, no2, ozone, pm10, pm25
weight_social_E = [0.3,0.2,0.3,0.2]                     # 인구밀도, 유동인구밀도, 충전기당 전기차 수, 도로보급률
weight_social_H = [0.3,0.2,0.3,0.2]                     # 인구밀도, 유동인구밀도, 충전기당 수소차 수, 도로보급률
weight_economic_E = [0.8,0.2]                           # 전기차충전기 설치비용, 평균소득
weight_economic_H = [0.4,0.2,0.4]                       # 수소차충전기 설치비용, 평균소득, 수소차충전소 기대수익
weight_technical_E = [[0.7,0.7,0.9,0.7,1.0]]            # 충전기용량
weight_technical_H = [[0.35,0.7],0.3]                   # 수소연료 공급방식, 최대 충전 가능한 수소차 수


## 최종 확률 결합 가중치
weight_total = [0.1, 0.4, 0.3, 0.2]     # 환경적 요인, 사회적 요인, 경제적 요인, 기술적 요인


print("환경변수(전기,수소)",variable_environment)
print("사회변수(전기)",variable_social_E)
print("사회변수(수소)",variable_social_H)
print("경제변수(전기)",variable_economic_E)
print("경제변수(수소)",variable_economic_H)
print("기술변수(전기)",variable_technical_E)
print("기술변수(수소)",variable_technical_H)

환경변수(전기,수소) [0.8287531282163694, 0.7911203783441816, 0.909590433996196, 0.8017202428389887, 0.8866811666335976, 0.8415688322677302]
사회변수(전기) [0.253606432432148, 0.5848617448890694, 0.7311285465884958, 0.6947193261807018]
사회변수(수소) [0.253606432432148, 0.5848617448890694, 1.0, 0.6947193261807018]
경제변수(전기) [0.5739632891115474, 0.22531111975287588]
경제변수(수소) [0.5379381229554602, 0.22531111975287588, 0.13766606094638267]
기술변수(전기) [[0.24346076458752516, 0.6197183098591549, 0.07042253521126761, 0.012072434607645875, 0.05432595573440644]]
기술변수(수소) [[0.3333333333333333, 0.6666666666666666], 0.9999638930949155]


### 결합확률(Joint Probability) 함수

In [146]:
a = list(zip(variable_technical_E,weight_technical_E))
print(a)

[([0.24346076458752516, 0.6197183098591549, 0.07042253521126761, 0.012072434607645875, 0.05432595573440644], [0.7, 0.7, 0.9, 0.7, 1.0])]


In [147]:
# 결합 확률 구하기
def joint_pro(variable, weight):
    result = 0.0
    variable_weight = list(zip(variable,weight))
    for pro, wei in variable_weight:
        # 이중리스트인 경우
        if isinstance(pro,list) and isinstance(wei,list):
            max = 0.0
            value_w = -1
            v_w = list(zip(pro,wei))
            for p,w in v_w:
                if max <= p:
                    max = p
                    value_w = w
            result += max*value_w
            
        else:
            result += pro*wei
    return result

In [148]:
# 상위 요소 결합 확률
environment_joint_pro = round(joint_pro(variable_environment,weight_environment),6)
social_joint_pro_E = round(joint_pro(variable_social_E,weight_social_E),6)
social_joint_pro_H = round(joint_pro(variable_social_H,weight_social_H),6)
economic_joint_pro_E = round(joint_pro(variable_economic_E,weight_economic_E),6)
economic_joint_pro_H = round(joint_pro(variable_economic_H,weight_economic_H),6)
technical_joint_pro_E = round(joint_pro(variable_technical_E,weight_technical_E),6)
technical_joint_pro_H = round(joint_pro(variable_technical_H,weight_technical_H),6)

print("Joint Probability")
print(environment_joint_pro, "/", round(1-environment_joint_pro,6))
print(social_joint_pro_E,"/", round(1-social_joint_pro_E,6))
print(social_joint_pro_H,"/", round(1-social_joint_pro_H,6))
print(economic_joint_pro_E,"/", round(1-economic_joint_pro_E,6))
print(economic_joint_pro_H,"/", round(1-economic_joint_pro_H,6))
print(technical_joint_pro_E,"/", round(1-technical_joint_pro_E,6))
print(technical_joint_pro_H,"/", round(1-technical_joint_pro_H,6))

Joint Probability
0.860383 / 0.139617
0.551337 / 0.448663
0.631998 / 0.368002
0.504233 / 0.495767
0.315304 / 0.684696
0.433803 / 0.566197
0.766656 / 0.233344


### 조건부확률(Conditional Probability) 분포표

 - 환경적 요인

    오존, 아황산가스, 이산화질소, 일산화탄소, 미세먼지(pm10, pm25)

 - 사회적 요인
   
    고정인구수, 유동인구 수, 친환경 자동차 등록대수, LPG 충전소 수, 충전소 수(전기,수소), 교통 편의성, 주변 충전소와의 거리, 고속도로 여부

 - 경제적 요인
   
    전기차 충전기 설치 비용, 수소 충전소 구축 비용, LPG 충전소 토지비용, 총 유지비용, 주차 구획 수

 - 기술적 요소

    복합 충전소 여부(수소), 충전시간에 따른 급속/완속 충전 방식

In [ ]:
# 결합 확률 구하기
def joint_pro(variable, weight):
    result = 0.0
    variable_weight = list(zip(variable,weight))
    for pro, wei in variable_weight:
        # 이중리스트인 경우
        if isinstance(pro,list) and isinstance(wei,list):
            max = 0.0
            value_w = -1
            v_w = list(zip(pro,wei))
            for p,w in v_w:
                if max <= p:
                    max = p
                    value_w = w
            result += max*value_w
            
        else:
            result += pro*wei
    return result

In [144]:
# 조건부 확률 구하기
def conditional_pro(weight):
    '''
    weight : 결합할 변수의 가중치 값(list)
    '''
    weight_len = 0
    table_len = 1
    for wei in weight:
        if isinstance(wei,list):    # 상태가 3개 이상인 정성적인 변수의 가중치인 경우
            for i in wei:
                weight_len += 1
            # 조건부확률테이블의 길이는 (변수 cardinality들의 곱)의 크기를 가짐
            table_len *= weight_len
        else:
            table_len *= 2
    
    # 결과 리스트 초기화
    result_True = [0.0]*table_len
    
    for i in range(len(weight)): # 3번
        # cardinality가 2인 변수의 가중치인 경우
        if not isinstance(wei,list):
            for j in range(table_len): # 8번
                # 변수의 T,F 거짓값
                cycle = table_len/(2**(i+1))
                if int(j/cycle)%2 == 0:
                    result_True[j] += weight[i]
                    # 부동소수점 오차 제거
                    result_True[j] = round(result_True[j],6)

    result_False = list(reversed(result_True))

    return result_True, result_False


In [145]:
# 상위 요인 조건부 확률
environment_pro, environment_pro_negative = conditional_pro(weight_environment)
social_E_pro, social_E_pro_negative = conditional_pro(weight_social_E)
social_H_pro, social_H_pro_negative = conditional_pro(weight_social_H)
economic_E_pro, economic_E_pro_negative = conditional_pro(weight_economic_E)
economic_H_pro, economic_H_pro_negative = conditional_pro(weight_economic_H)
technical_E_pro, technical_E_pro_negative = conditional_pro(weight_technical_E)
technical_H_pro, technical_H_pro_negative = conditional_pro(weight_technical_H)

print("환경적 요소 조건부 확률",'\n', environment_pro,'\n',environment_pro_negative)
print("사회적(전기) 요소 조건부 확률",'\n', social_E_pro,'\n',social_E_pro_negative)
print("사회적(수소) 요소 조건부 확률",'\n', social_H_pro,'\n',social_H_pro_negative)
print("경제적(전기) 요소 조건부 확률",'\n', economic_E_pro,'\n',economic_E_pro_negative)
print("경제적(수소) 요소 조건부 확률",'\n', economic_H_pro,'\n',economic_H_pro_negative)
print("기술적(전기) 요소 조건부 확률",'\n', technical_E_pro,'\n',technical_E_pro_negative)
print("기술적(수소) 요소 조건부 확률",'\n', technical_H_pro,'\n',technical_H_pro_negative)

TypeError: unsupported operand type(s) for +=: 'float' and 'list'

## 베이지안 네트워크 모델 구성

In [ ]:
from IPython.core.display import Image
from networkx.drawing.nx_pydot import to_pydot

from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel

In [ ]:
from pgmpy.inference import VariableElimination
from pgmpy.factors.discrete import DiscreteFactor
from pgmpy.models import FactorGraph

In [ ]:
### 전기차 요인
# 환경적 요인 하위 변수 확률
o3 = [[o3_pro], [1-o3_pro]]
so2 = [[so2_pro], [1-so2_pro]]
no2 = [[no2_pro], [1-no2_pro]]
co = [[co_pro], [1-co_pro]]
pm10 = [[pm10_pro], [1-pm10_pro]]
pm25 = [[pm25_pro], [1-pm25_pro]]

# 사회적 요인 하위 변수 확률
people = [[density_pro],[1-density_pro]]
floating = [[floating_pro],[1-floating_pro]]
EV = [[elec_vehicle_pro],[1-elec_vehicle_pro]]
EVCS = [[EVCS_pro],[1-EVCS_pro]]
intersection_E = [[intersection_E_pro],[1-intersection_E_pro]]
distance_E = [[EV_distance_pro],[1-EV_distance_pro]]
highway_E = [[highway_E_pro],[1-highway_E_pro]]

# 경제적 요인 하위 변수 확률
cost_E = [[EVCS_cost],[1-EVCS_cost]]
area = [[parking_area_pro],[1-parking_area_pro]]

# 기술적 요인 하위 변수 확률
type = [[charger_pro_fast],[1-charger_pro_fast]]


### 수소차 요인
# 환경적 요인은 전기차와 동일 -> 환경적 모델 하나로 구성

# 사회적 요인 하위 변수 확률
people = [[busan_people_pro],[1-busan_people_pro]]
floating = [[foottraffic_pro],[1-foottraffic_pro]]
LPG = [[LPG_pro],[1-LPG_pro]]
HV = [[hidro_vehicle_pro],[1-hidro_vehicle_pro]]
HVCS = [[HVCS_pro],[1-HVCS_pro]]
intersection_H = [[intersection_H_pro],[1-intersection_H_pro]]
distance_H = [[HV_distance_pro],[1-HV_distance_pro]]
highway_H = [[highway_H_pro],[1-highway_H_pro]] 

# 경제적 요인 하위 변수 확률
cost_H = [[HVCS_construction_cost_pro],[1-HVCS_construction_cost_pro]]
LPG_land = [[LPG_land_pro],[1-LPG_land_pro]]

# 기술적 요인 하위 변수 확률
complex = [[complex_pro],[1-complex_pro]]

environment_model = BayesianModel([('o3', 'environment'), ('so2', 'environment'), ('no2', 'environment'), ('co', 'environment'),
                                   ('pm10', 'environment'), ('pm25', 'environment'), ('khai', 'environment')])
social_model_E = BayesianModel([('people','social_E'),('floating','social_E'),('EV','social_E'),('EVCS','social_E'),
                                ('intersection_E','social_E'),('distance_E','social_E'),('highway_E','social_E')])
social_model_H = BayesianModel([('people','social_H'),('floating','social_H'),('LPG','social_H'),('HV','social_H'),('HVCS','social_H'),
                                ('intersection_H','social_H'),('distance_H','social_H'),('highway_H','social_H')])
economic_model_E = BayesianModel([('cost_E','economic_E'),('area','economic_E')])
economic_model_H = BayesianModel([('cost_H','economic_H'),('LPG_land','economic_H')])
technical_model_E = BayesianModel([('type','technical_E')])
technical_model_H = BayesianModel([('complex','technical_H')])


### 환경적 요인의 하위 변수 모델링 
P_o3 = TabularCPD('o3', 2,values=o3)
P_so2 = TabularCPD('so2', 2,values=so2)
P_no2 = TabularCPD('no2', 2,values=no2)
P_co = TabularCPD('co', 2,values=co)
P_pm10 = TabularCPD('pm10', 2,values=pm10)
P_pm25 = TabularCPD('pm25', 2,values=pm25)
P_khai = TabularCPD('khai',2,values=khai)
P_environment_I_o3_so2_no2_co_pm10_pm25_khai = TabularCPD('environment',2,[environment_pro,environment_pro_negative],
                                                            evidence=['o3','so2','no2','co','pm10','pm25','khai'],
                                                            evidence_card=[2,2,2,2,2,2,2]
                                                            )

environment_model.add_cpds(P_o3,P_so2,P_no2,P_co,P_pm10,P_pm25,P_khai, P_environment_I_o3_so2_no2_co_pm10_pm25_khai)                                


### 사회적 요인의 하위 변수 모델링(전기)
P_people = TabularCPD('people',2,values=people)
P_floating = TabularCPD('floating',2,values=floating)
P_EV = TabularCPD('EV',2,values=EV)
P_EVCS = TabularCPD('EVCS',2,values=EVCS)
P_intersection_E = TabularCPD('intersection_E',2,values=intersection_E)
P_distance_E = TabularCPD('distance_E',2,values=distance_E)
P_highway_E = TabularCPD('highway_E',2,values=highway_E)
P_social_E_I_people_floating_EV_EVCS_intersection_E_distance_E_highway_E = TabularCPD('social_E',2,[social_E_pro,social_E_pro_negative],
                                                                                        evidence=['people','floating','EV','EVCS',
                                                                                                  'intersection_E','distance_E','highway_E'],
                                                                                        evidence_card=[2,2,2,2,2,2,2])

social_model_E.add_cpds(P_people,P_floating,P_EV,P_EVCS,P_intersection_E,P_distance_E,P_highway_E,P_social_E_I_people_floating_EV_EVCS_intersection_E_distance_E_highway_E)


### 사회적 요인의 하위 변수 모델링(수소)
P_people = TabularCPD('people',2,values=people)
P_floating = TabularCPD('floating',2,values=floating)
P_LPG = TabularCPD('LPG',2,values=LPG)
P_HV = TabularCPD('HV',2,values=HV)
P_HVCS = TabularCPD('HVCS',2,values=HVCS)
P_intersection_H = TabularCPD('intersection_H',2,values=intersection_H)
P_distance_H = TabularCPD('distance_H',2,values=distance_H)
P_highway_H = TabularCPD('highway_H',2,values=highway_H)
P_social_H_I_people_floating_LPG_HV_HVCS_intersection_H_distance_H_highway_H = TabularCPD('social_H',2,[social_H_pro,social_H_pro_negative],
                                                                                            evidence=['people','floating','LPG','HV',
                                                                                                      'HVCS','intersection_H','distance_H','highway_H'],
                                                                                            evidence_card=[2,2,2,2,2,2,2,2])

social_model_H.add_cpds(P_people,P_floating,P_LPG,P_HV,P_HVCS,P_intersection_H,P_distance_H,P_highway_H,P_social_H_I_people_floating_LPG_HV_HVCS_intersection_H_distance_H_highway_H) 

### 경제적 요인의 하위 변수 모델링(전기)
P_cost_E = TabularCPD('cost_E',2,values=cost_E)
P_area = TabularCPD('area',2,values=area)
P_economic_E_I_cost_E_area = TabularCPD('economic_E',2,[economic_E_pro,economic_E_pro_negative],
                                        evidence=['cost_E','area'],
                                        evidence_card=[2,2])

economic_model_E.add_cpds(P_cost_E,P_area,P_economic_E_I_cost_E_area)   

### 경제적 요인의 하위 변수 모델링(수소)
P_cost_H = TabularCPD('cost_H',2,values=cost_H)
P_LPG_land = TabularCPD('LPG_land',2,values=LPG_land)
P_economic_H_I_cost_H_LPG_land = TabularCPD('economic_H',2,[economic_H_pro,economic_H_pro_negative],
                                            evidence=['cost_H','LPG_land'],
                                            evidence_card=[2,2])

economic_model_H.add_cpds(P_cost_H,P_LPG_land,P_economic_H_I_cost_H_LPG_land)       

### 기술적 요인의 하위 변수 모델링(전기)
P_type = TabularCPD('type',2,values=type)
P_technical_E_I_type = TabularCPD('technical_E',2,[technical_E_pro,technical_E_pro_negative],
                            evidence=['type'],
                            evidence_card=[2])

technical_model_E.add_cpds(P_type,P_technical_E_I_type)


### 기술적 요인의 하위 변수 모델링(수소)
P_complex = TabularCPD('complex',2,values=complex)
P_technical_H_I_complex = TabularCPD('technical_H',2,[technical_H_pro,technical_H_pro_negative],
                                    evidence=['complex'],
                                    evidence_card=[2])

technical_model_H.add_cpds(P_complex,P_technical_H_I_complex)


infer_environment = VariableElimination(environment_model)
infer_social_E = VariableElimination(social_model_E)
infer_social_H = VariableElimination(social_model_H)
infer_economic_E = VariableElimination(economic_model_E)
infer_economic_H = VariableElimination(economic_model_H)
infer_technical_E = VariableElimination(technical_model_E)
infer_technical_H = VariableElimination(technical_model_H)

c:\Users\SAMSUNG\anaconda3\envs\GVCS\lib\site-packages\pgmpy\models\BayesianModel.py:8: FutureWarning:

BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.



In [ ]:
### 전기차
# 조건부확률 분포표
print(P_environment_I_o3_so2_no2_co_pm10_pm25_khai)                             # 환경적 요인
print(P_social_E_I_people_floating_EV_EVCS_intersection_E_distance_E_highway_E) # 사회적 요인
print(P_economic_E_I_cost_E_area)                                               # 경제적 요인
print(P_technical_E_I_type)                                                     # 기술적 요인

+----------------+---------+-----+---------+---------+---------+
| o3             | o3(0)   | ... | o3(1)   | o3(1)   | o3(1)   |
+----------------+---------+-----+---------+---------+---------+
| so2            | so2(0)  | ... | so2(1)  | so2(1)  | so2(1)  |
+----------------+---------+-----+---------+---------+---------+
| no2            | no2(0)  | ... | no2(1)  | no2(1)  | no2(1)  |
+----------------+---------+-----+---------+---------+---------+
| co             | co(0)   | ... | co(1)   | co(1)   | co(1)   |
+----------------+---------+-----+---------+---------+---------+
| pm10           | pm10(0) | ... | pm10(1) | pm10(1) | pm10(1) |
+----------------+---------+-----+---------+---------+---------+
| pm25           | pm25(0) | ... | pm25(0) | pm25(1) | pm25(1) |
+----------------+---------+-----+---------+---------+---------+
| khai           | khai(0) | ... | khai(1) | khai(0) | khai(1) |
+----------------+---------+-----+---------+---------+---------+
| environment(0) | 1.0   

In [ ]:
# 환경적요인의 상태(True:0)를 아는 경우, 각 하위 요인의 확률(True:0,False:1)
print(infer_environment.query(["o3"], evidence={"environment":0},show_progress=False))
print(infer_environment.query(["so2"], evidence={"environment":0},show_progress=False))
print(infer_environment.query(["no2"], evidence={"environment":0},show_progress=False))
print(infer_environment.query(["co"], evidence={"environment":0},show_progress=False))
print(infer_environment.query(["pm10"], evidence={"environment":0},show_progress=False))
print(infer_environment.query(["pm25"], evidence={"environment":0},show_progress=False))
print(infer_environment.query(["khai"], evidence={"environment":0},show_progress=False))

+-------+-----------+
| o3    |   phi(o3) |
+=======+===========+
| o3(0) |    0.8264 |
+-------+-----------+
| o3(1) |    0.1736 |
+-------+-----------+
+--------+------------+
| so2    |   phi(so2) |
+========+============+
| so2(0) |     0.8167 |
+--------+------------+
| so2(1) |     0.1833 |
+--------+------------+
+--------+------------+
| no2    |   phi(no2) |
+========+============+
| no2(0) |     0.9223 |
+--------+------------+
| no2(1) |     0.0777 |
+--------+------------+
+-------+-----------+
| co    |   phi(co) |
+=======+===========+
| co(0) |    0.8508 |
+-------+-----------+
| co(1) |    0.1492 |
+-------+-----------+
+---------+-------------+
| pm10    |   phi(pm10) |
+=========+=============+
| pm10(0) |      0.9023 |
+---------+-------------+
| pm10(1) |      0.0977 |
+---------+-------------+
+---------+-------------+
| pm25    |   phi(pm25) |
+=========+=============+
| pm25(0) |      0.8622 |
+---------+-------------+
| pm25(1) |      0.1378 |
+---------+-------

In [ ]:
### 전기차
# 하위요인의 상태를 모르는 경우, 상위 요인의 상태(적합:0, 부적합:1)에 대한 확률
print(infer_environment.query(["environment"],show_progress=False))
print(infer_social_E.query(["social_E"],show_progress=False))
print(infer_economic_E.query(["economic_E"],show_progress=False))
print(infer_technical_E.query(["technical_E"],show_progress=False))

+----------------+--------------------+
| environment    |   phi(environment) |
+================+====================+
| environment(0) |             0.8386 |
+----------------+--------------------+
| environment(1) |             0.1614 |
+----------------+--------------------+
+-------------+-----------------+
| social_E    |   phi(social_E) |
+=============+=================+
| social_E(0) |          0.4154 |
+-------------+-----------------+
| social_E(1) |          0.5846 |
+-------------+-----------------+
+---------------+-------------------+
| economic_E    |   phi(economic_E) |
+===============+===================+
| economic_E(0) |            0.4381 |
+---------------+-------------------+
| economic_E(1) |            0.5619 |
+---------------+-------------------+
+----------------+--------------------+
| technical_E    |   phi(technical_E) |
+================+====================+
| technical_E(0) |             0.3800 |
+----------------+--------------------+
| technical_E(1)

In [ ]:
# 하위 요인(o3, no2)의 상태(o3:적합, no2:부적합)가 밝혀진 경우, 환경적 요인의 상태에 대한 확률
result = infer_environment.query(["environment"], evidence={"o3":0, "no2":1}, show_progress=False)
print(result)
# 환경적 요인이 적합할 확률
result.get_value(environment=0)

+----------------+--------------------+
| environment    |   phi(environment) |
+================+====================+
| environment(0) |             0.7461 |
+----------------+--------------------+
| environment(1) |             0.2539 |
+----------------+--------------------+


0.7461023537134321

## 입지에 대한 확률 업데이트

#### 관측된 데이트에 따른 확률 업데이트(전기차)

In [ ]:
def parking_update(name, infer_environment, infer_social_E, infer_economic_E, infer_technical_E):
    parking_area = df_parking[df_parking['주차장명']==name]['주차구획수'].values[0]
    parking_distance = df_parking[df_parking['주차장명']==name]['주변충전소 최단거리'].values[0]
    parking_highway= df_parking[df_parking['주차장명']==name]['주변고속도로까지 최단거리'].values[0]
    parking_intersection = df_parking[df_parking['주차장명']==name]['교차로'].values[0]
    gu = df_parking[df_parking['주차장명']==name]['행정구'].values[0]  
    parking_people = df_busan_people.loc[gu].values[0] 
    parking_floating = df_foottraffic.loc[gu].values[0]
    charger_pro_fast,charger_pro_standard = charge_type(parking_people,parking_floating)
    type = [[charger_pro_fast],[charger_pro_standard]]

    ### 관측된 수치를 기준치로 적합성 판단
    # 0:적합, 1:부적합
    if parking_area > parking_area_standard:
        area_state = 0 
    else:
        area_state = 1
    
    if parking_distance > EV_distance_standard:
        distance_E_state = 0 
    else:
        distance_E_state = 1

    if parking_highway > highway_standard_E:
        highway_E_state = 1 
    else:
        highway_E_state = 0
    
    if parking_intersection > intersection_standard:
        intersection_E_state = 0 
    else:
        intersection_E_state = 1
    
    if parking_people > busan_people_standard:
        people_state = 0 
    else:
        people_state = 1
        
    if parking_floating > foottraffic_standard:
        floating_state = 0 
    else:
        floating_state = 1

    ### 기술적 요인의 하위 변수 업데이트(전기)
    technical_model_E = BayesianModel([('type','technical_E')])

    P_type = TabularCPD('type',2,values=type)
    P_technical_E_I_type = TabularCPD('technical_E',2,[technical_E_pro,technical_E_pro_negative],
                                evidence=['type'],
                                evidence_card=[2])

    technical_model_E.add_cpds(P_type,P_technical_E_I_type)
    infer_technical_E = VariableElimination(technical_model_E)
    
    result_env = infer_environment.query(["environment"],show_progress=False)
    result_soc = infer_social_E.query(["social_E"],evidence={"distance_E":distance_E_state,"highway_E":highway_E_state,"intersection_E":intersection_E_state,
                                                "people":people_state,"floating":floating_state}, show_progress=False)
    result_eco = infer_economic_E.query(["economic_E"],evidence={"area":area_state},show_progress=False)
    result_tec = infer_technical_E.query(["technical_E"],show_progress=False)

    pro_env = result_env.get_value(environment=0)
    pro_soc = result_soc.get_value(social_E=0)
    pro_eco = result_eco.get_value(economic_E=0)
    pro_tec = result_tec.get_value(technical_E=0)

    return pro_env, pro_soc, pro_eco, pro_tec

In [ ]:
# parking_list = df_parking['주차장명'].tolist()
# list_env_E = []
# list_soc_E = []
# list_eco_E = []
# list_tec_E = []
# list_total_E = []
# for place in parking_list:
#     pro_env_E, pro_soc_E, pro_eco_E, pro_tec_E = parking_update(place, infer_environment, infer_social_E, infer_economic_E, infer_technical_E)
#     list_env_E.append(round(pro_env_E,6))
#     list_soc_E.append(round(pro_soc_E,6))
#     list_eco_E.append(round(pro_eco_E,6))
#     list_tec_E.append(round(pro_tec_E,6))
#     # 가중치를 통해 최종확률 도출
#     total_pro = 0
#     total_pro += pro_env_E * weight_total[0]
#     total_pro += pro_soc_E * weight_total[1]
#     total_pro += pro_eco_E * weight_total[2]
#     total_pro += pro_tec_E * weight_total[3]
#     list_total_E.append(round(total_pro,6))

In [ ]:
# df_parking.insert(14,'기술적요인',list_tec_E)
# df_parking.insert(14,'경제적요인',list_eco_E)
# df_parking.insert(14,'사회적요인',list_soc_E)
# df_parking.insert(14,'환경적요인',list_env_E)
# df_parking.insert(18,'최종확률',list_total_E)
# df_parking.to_csv(parking_file_path,encoding='cp949')

#### 관측된 데이터에 대한 확률 업데이트(수소차)

In [ ]:
def LPG_update(name, infer_environment, infer_social_H, infer_economic_H, infer_technical_H):
    gu = df_LPG[df_LPG['업소명']==name]['행정구'].values[0]                              
    LPG_people = df_busan_people.loc[gu].values[0] 
    LPG_floating = df_foottraffic.loc[gu].values[0]
    LPG_land = df_LPG[df_LPG['업소명']==name]['토지비용'].values[0]                    
    LPG_distance = df_LPG[df_LPG['업소명']==name]['주변충전소 최단거리'].values[0]     
    LPG_highway = df_LPG[df_LPG['업소명']==name]['주변고속도로 최단거리'].values[0]    
    LPG_intersection = df_LPG[df_LPG['업소명']==name]['교차로 수'].values[0]     
    LPG_type = charge_type(LPG_people,LPG_floating)

    # 기존 LPG 충전소 위경도 추출
    LPG_lat = df_LPG['위도'].tolist()
    LPG_lon = df_LPG['경도'].tolist()
    location_LPG = list(zip(LPG_lat,LPG_lon))
    location_place = (df_LPG[df_LPG['업소명']==name]['위도'].values[0], df_LPG[df_LPG['업소명']==name]['경도'].values[0]) 
    if location_place in location_LPG:
        complex_pro = 1.0
    else:
        complex_pro = 0.0
    
    ### 관측된 수치를 기준치로 적합성 판단
    # 0:적합, 1:부적합
    if LPG_people > busan_people_standard:
        people_state = 0
    else:
        people_state = 1
    
    if LPG_floating > foottraffic_standard:
        floating_state = 0
    else:
        floating_state = 1
    
    if LPG_land > LPG_land_standard:
        land_state = 1
    else:
        land_state = 0

    if LPG_distance > HV_distance_standard:
        distance_state = 0
    else:
        distance_state = 1
    
    if LPG_highway > highway_standard_H:
        highway_state = 1
    else:
        highway_state = 0
    
    if LPG_intersection > intersection_standard:
        intersection_state = 0
    else:
        intersection_state = 1

    if complex_pro == 1.0:
        complex_state = 0
    else: 
        complex_state = 1
    
    result_env = infer_environment.query(["environment"],show_progress=False)
    result_soc = infer_social_H.query(["social_H"],evidence={"distance_H":distance_state,"highway_H":highway_state,"intersection_H":intersection_state,
                                                             "people":people_state,"floating":floating_state}, show_progress=False)
    result_eco = infer_economic_H.query(["economic_H"],evidence={"LPG_land":land_state},show_progress=False)
    result_tec = infer_technical_H.query(["technical_H"],evidence={"complex":complex_state},show_progress=False)

    pro_env = result_env.get_value(environment=0)
    pro_soc = result_soc.get_value(social_H=0)
    pro_eco = result_eco.get_value(economic_H=0)
    pro_tec = result_tec.get_value(technical_H=0)

    return pro_env, pro_soc, pro_eco, pro_tec

In [ ]:
# LPG_list = df_LPG['업소명'].tolist()
# list_env_H = []
# list_soc_H = []
# list_eco_H = []
# list_tec_H = []
# list_total_H = []
# for place in LPG_list:
#     pro_env_H, pro_soc_H, pro_eco_H, pro_tec_H = LPG_update(place, infer_environment, infer_social_H, infer_economic_H, infer_technical_H)
#     list_env_H.append(round(pro_env_H,6))
#     list_soc_H.append(round(pro_soc_H,6))
#     list_eco_H.append(round(pro_eco_H,6))
#     list_tec_H.append(round(pro_tec_H,6))
#     # 가중치를 통해 최종확률 도출
#     total_pro = 0
#     total_pro += pro_env_H * weight_total[0]
#     total_pro += pro_soc_H * weight_total[1]
#     total_pro += pro_eco_H * weight_total[2]
#     total_pro += pro_tec_H * weight_total[3]
#     list_total_H.append(round(total_pro,6))

In [ ]:
# df_LPG.insert(13,'기술적요인',list_tec_H)
# df_LPG.insert(13,'경제적요인',list_eco_H)
# df_LPG.insert(13,'사회적요인',list_soc_H)
# df_LPG.insert(13,'환경적요인',list_env_H)
# df_LPG.insert(17,'최종확률',list_total_H)
# df_LPG.to_csv(LPG_file_path,encoding='cp949')

In [ ]:
df_parking

,운영형태,행정구,관리기관명,주차장명,주차장관리번호,소재지도로명주소,주차장구분,주차장유형,주차구획수,위도,경도,교차로,주변고속도로까지 최단거리,주변충전소 최단거리,환경적요인,사회적요인,경제적요인,기술적요인,최종확률
0,구(군)직영,사상구,부산광역시 사상구청,감전동주변,2010000013,사상구 감전동 589-2,공영,노외,26,35.154108,128.980071,239,4346.343726,0.061228,0.838573,0.643981,0.252453,0.488554,0.532397
1,민간위탁,사상구,부산광역시 사상구청,엄궁초등학교지하주차장,2010000016,사상구 엄궁로191번길 23,공영,노외,93,35.128446,128.973403,85,7240.940505,0.112529,0.838573,0.493981,0.252453,0.488554,0.472397
2,구(군)직영,사상구,부산광역시 사상구청,일신주택주변,2011000003,사상구 주례3동 545-3,공영,노외,10,35.145207,129.000672,165,5584.115172,0.247823,0.838573,0.643981,0.252453,0.488554,0.532397
3,구(군)직영,사상구,부산광역시 사상구청,주례중학교주변,2012000004,사상구 주례2동 82-13,공영,노외,8,35.154384,129.011131,276,5040.598424,0.094006,0.838573,0.643981,0.252453,0.488554,0.532397
4,민간위탁,사상구,부산광역시 사상구청,모라1동행정복지센터주변,2007001725,사상구 백양대로 949-6,공영,노상,156,35.171044,128.988085,249,2512.034932,0.190308,0.838573,0.643981,0.752453,0.488554,0.682397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,민간위탁,중구,부산광역시 중구청,중앙동 노외공영주차장,2009000002,중구 대청로135번길 36,공영,노외,29,35.104229,129.034810,593,10976.446670,0.143217,0.838573,0.443981,0.252453,0.393172,0.438090
469,민간운영,기장군,부산광역시 기장군청,1구역 2소,2008000002,기장군 기장읍 280-2,공영,노상,12,35.222141,129.206947,19,162.104257,1.603011,0.838573,0.493981,0.252453,0.327783,0.448282
470,민간운영,기장군,부산광역시 기장군청,1구역 3소,2008000003,기장군 기장읍 444-1,공영,노상,13,35.222141,129.206947,19,162.104257,1.603011,0.838573,0.493981,0.252453,0.327783,0.448282
471,민간운영,기장군,부산광역시 기장군청,3구역 1소,2008000006,기장군 기장읍 418-12,공영,노상,24,35.222141,129.206947,19,162.104257,1.603011,0.838573,0.493981,0.252453,0.327783,0.448282


In [ ]:
df_LPG

,행정시,행정구,업소명,소재지,관리구분,위도,경도,면적,공시지가,토지비용,교차로 수
0,부산,강서구,개인택시신공항충전소,부산 강서구 경전철로188번길 95,자동차충전,35.198669,128.966594,2545.0,2144000.0,5.456480e+09,52
1,부산,강서구,부산공항LPG충전소,부산 강서구 공항로 459,자동차충전,35.149064,128.951891,3105.0,1174000.0,3.645270e+09,16
2,부산,강서구,신항만에너지,부산 강서구 낙동남로 507,자동차충전,35.112947,128.875260,3105.0,1174000.0,3.645270e+09,76
3,부산,강서구,르노삼성자동차(주),부산 강서구 르노삼성대로 61,자동차충전,35.095408,128.883215,1538.0,640100.0,9.844738e+08,82
4,부산,강서구,죽림LPG충전소,부산 강서구 호계로 102,자동차충전,35.200736,128.893462,1580.0,1017000.0,1.606860e+09,16
5,부산,강서구,가락충전소,부산 강서구 호계로 195,자동차충전,35.208847,128.890148,2600.0,1017000.0,2.644200e+09,20
6,부산,금정구,(주)해연,부산 금정구 개좌로 225-15,용기+탱크+13kg용기,35.230796,129.122388,3346.0,1370000.0,4.584020e+09,119
7,부산,금정구,(주)신진가스,부산 금정구 반송로 433,자동차충전,35.215006,129.111143,1646.4,2797000.0,4.604981e+09,407
8,부산,기장군,마라톤가스(주)청강충전소,부산 기장군 기장읍 기장대로 343 A동,자동차충전,35.225657,129.213988,2592.0,1069000.0,2.770848e+09,45
9,부산,기장군,한국에너지산업(주),부산 기장군 기장읍 기장대로413번길 38,용기+자동차+탱크+13kg용기,35.235543,129.215629,2473.0,1063000.0,2.628799e+09,241
